In [ ]:
!pip install transformers
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━

# Messing around with ChemBERTa for fun and for education

The first half of this colab is just fun experiments trying to understand ChemBERTa and it's tokenizer better.

In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer, pipeline, RobertaModel, RobertaTokenizer


model = AutoModelForMaskedLM.from_pretrained("seyonec/PubChem10M_SMILES_BPE_450k")
tokenizer = AutoTokenizer.from_pretrained("seyonec/PubChem10M_SMILES_BPE_450k")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/336M [00:00<?, ?B/s]

Some weights of the model checkpoint at seyonec/PubChem10M_SMILES_BPE_450k were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/165k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/101k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Let's print the tokenizer and model's stats

In [ ]:
print(tokenizer)

RobertaTokenizerFast(name_or_path='seyonec/PubChem10M_SMILES_BPE_450k', vocab_size=7924, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	4: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True, special=True),
}


In [ ]:
print(model.config)

RobertaConfig {
  "_name_or_path": "seyonec/PubChem10M_SMILES_BPE_450k",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.44.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 52000
}



One bizarre thing to note here: The default tokenizer has a smaller vocab_size than the vocab_size of the model. What's going on? Not sure, but I think it's because the vocab_size of the model is based off usual language, where as the tokenizer is much smaller since it only needs to cover all possible SMILES. It's weird though, that they didn't adjust the vocab_size in the model. I guess that the weights for last 52000-7924 vocabs just don't count. Seems super wasteful.

### Names of all the weights in ChemBERTa


In [ ]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"{name} has shape {param.shape}")

roberta.embeddings.word_embeddings.weight has shape torch.Size([52000, 768])
roberta.embeddings.position_embeddings.weight has shape torch.Size([512, 768])
roberta.embeddings.token_type_embeddings.weight has shape torch.Size([1, 768])
roberta.embeddings.LayerNorm.weight has shape torch.Size([768])
roberta.embeddings.LayerNorm.bias has shape torch.Size([768])
roberta.encoder.layer.0.attention.self.query.weight has shape torch.Size([768, 768])
roberta.encoder.layer.0.attention.self.query.bias has shape torch.Size([768])
roberta.encoder.layer.0.attention.self.key.weight has shape torch.Size([768, 768])
roberta.encoder.layer.0.attention.self.key.bias has shape torch.Size([768])
roberta.encoder.layer.0.attention.self.value.weight has shape torch.Size([768, 768])
roberta.encoder.layer.0.attention.self.value.bias has shape torch.Size([768])
roberta.encoder.layer.0.attention.output.dense.weight has shape torch.Size([768, 768])
roberta.encoder.layer.0.attention.output.dense.bias has shape torch

## Default inference pipeline in ChemBERTa

ChemBERTa is an LLM trained on the masked token task. Let's see how well it does it with an example. Below is an example of a masked smiles and what it "should" be. The fill-mask pipeline gives ChemBERTa's top 5 guesses

In [ ]:
fill_mask = pipeline('fill-mask', model=model, tokenizer=tokenizer)

smiles_mask = "C1=CC=CC<mask>C1"
smiles = "C1=CC=CC=C1"

masked_smi = fill_mask(smiles_mask)

for smi in masked_smi:
  print(smi)

{'score': 0.9755934476852417, 'token': 33, 'token_str': '=', 'sequence': 'C1=CC=CC=C1'}
{'score': 0.020923908799886703, 'token': 7, 'token_str': '#', 'sequence': 'C1=CC=CC#C1'}
{'score': 0.0007658947724848986, 'token': 21, 'token_str': '1', 'sequence': 'C1=CC=CC1C1'}
{'score': 0.00041297602001577616, 'token': 22, 'token_str': '2', 'sequence': 'C1=CC=CC2C1'}
{'score': 0.00025319133419543505, 'token': 352, 'token_str': '=[', 'sequence': 'C1=CC=CC=[C1'}


Pretty good! It gave a probability of 97% to the correct SMILES

## How to interpret logits for other tokens?

When a token isn't masked, what should you make of the logits/"probability" for it? For each token there a logit/prob for each of the possible 52000 tokens that could have been there. One way to intepret it is as if that token was actually masked, and the model is giving probabilities for all the tokens that could potentially go there. Another way to think of it is that it's about how "natural" this token is in this spot. If the model gives the token a high score for itself, it expected to see it there. Since we are giving the model a very natural sequence, we'd expect that it's scores for each of the tokens is highest for the actual token. Let's see.


In [ ]:
import torch

sequence = f"C1=CC=CC=CC1"
input = tokenizer.encode(sequence, return_tensors="pt")

for i in range(len(input[0])):
    decode = tokenizer.decode(input[0][i])
    encode = input[0][i]
    print(f"{decode} is the token {encode}")

token_logits = model(input)[0]
print(f"token logits shape {token_logits.shape}")

for i, token_id in enumerate(input[0]):

    print(f"token_id is {token_id}")
    # Get the logits for the i-th position
    logits_for_token_position = token_logits[0, i, :]  # Shape: [52000], all logits for this position
    probability_for_token_position = torch.softmax(logits_for_token_position, dim=0)

    logit_for_correct_token = logits_for_token_position[token_id]
    prob_for_correct_token = probability_for_token_position[token_id]
    print(f"The probability of the correct token is {round(prob_for_correct_token.item(),2)}")

    # Find the maximum logit value at this position
    max_logit = logits_for_token_position.max().item()

    # Check if the logit for the actual token is the highest
    if logit_for_correct_token == max_logit:
         print(f"Position {i}: Correct: {token_id} has the highest logit.")
    else:
        print(f"Position {i}: Incorrect: {token_id} does NOT have the highest logit.")

<s> is the token 0
C is the token 39
1 is the token 21
= is the token 33
CC is the token 262
= is the token 33
CC is the token 262
= is the token 33
CC is the token 262
1 is the token 21
</s> is the token 2
token logits shape torch.Size([1, 11, 52000])
token_id is 0
The probability of the correct token is 0.0
Position 0: Incorrect: 0 does NOT have the highest logit.
token_id is 39
The probability of the correct token is 0.97
Position 1: Correct: 39 has the highest logit.
token_id is 21
The probability of the correct token is 0.99
Position 2: Correct: 21 has the highest logit.
token_id is 33
The probability of the correct token is 0.99
Position 3: Correct: 33 has the highest logit.
token_id is 262
The probability of the correct token is 0.86
Position 4: Correct: 262 has the highest logit.
token_id is 33
The probability of the correct token is 1.0
Position 5: Correct: 33 has the highest logit.
token_id is 262
The probability of the correct token is 0.86
Position 6: Correct: 262 has the h

The only two it got incorrect where the start and end, which is fair enough.

## Testing it out on masked sequences again- this time without the pipeline

In [ ]:
import torch.nn.functional as F
import torch

sequence = f"C1=CC=CC={tokenizer.mask_token}1"
input = tokenizer.encode(sequence, return_tensors="pt")
mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]

mask_token_probs = F.softmax(token_logits[0, mask_token_index, :][0], dim=0)
top_5_tokens = torch.topk(mask_token_probs, 5).indices.tolist()

for token in top_5_tokens:
  smi = (sequence.replace(tokenizer.mask_token, tokenizer.decode([token])))
  print (f"{smi} has a probability of {round(mask_token_probs[token].item(),2)}%")


C1=CC=CC=CC1 has a probability of 0.83%
C1=CC=CC=CNC1 has a probability of 0.04%
C1=CC=CC=COC1 has a probability of 0.03%
C1=CC=CC=CN1 has a probability of 0.01%
C1=CC=CC=CCC1 has a probability of 0.01%


## Embeddings from ChemBERTa

Each token in a SMILES sequence is embedded as a vector of dimension hidden_dimension. Then the model updates this embedding at every layer. In the last layer (before the big projection matrix back to vocab_size) the embedding is hopefully very rich and meaningful. The transformer model should have encoded important relationships between parts of the molecule into these vectors. That's why we'd like to be able to access these and use them in later parts of the model.

In [ ]:
sequence = f"C1=CC=CC=CC1"
inputs = tokenizer(sequence, return_tensors="pt")
outputs = model(**inputs, output_hidden_states=True)
hidden_states = outputs.hidden_states
print(f"The number of hidden states is {len(hidden_states)}, because there's one after each layer (except the last)")
print(f"The shape of the hidden states is {hidden_states[0].shape}")
number_of_tokens = hidden_states[0].shape[1]
hidden_dimension = hidden_states[0].shape[2]

# Let's look at the embeddings in the 0th hidden state versus in the final
original_token_embeddings = hidden_states[0][0]
last_token_embeddings = hidden_states[-1][0]

for i in range(number_of_tokens):
    dot_product = torch.dot(original_token_embeddings[i], last_token_embeddings[i])
    print(f"The {i}th token has dot product between original and final of {round(dot_product.item(),2)}")


print(f"On the other hand, this is what the dot product of two random vectors looks like")
dot_product_count = 0
for i in range(1000):
    random_vector1 = torch.randn(hidden_dimension)
    random_vector2 = torch.randn(hidden_dimension)
    dot_product_count += torch.abs(torch.dot(random_vector1, random_vector2)).item()

print(f"The average absolute value of the dot product of 100 random vectors is {round(dot_product_count/1000,2)}")

The number of hidden states is 7, because there's one after each layer (except the last)
The shape of the hidden states is torch.Size([1, 11, 768])
The 0th token has dot product between original and final of 39.41
The 1th token has dot product between original and final of 39.53
The 2th token has dot product between original and final of 114.06
The 3th token has dot product between original and final of 59.25
The 4th token has dot product between original and final of 12.53
The 5th token has dot product between original and final of 73.68
The 6th token has dot product between original and final of -13.18
The 7th token has dot product between original and final of 83.18
The 8th token has dot product between original and final of -1.96
The 9th token has dot product between original and final of 89.14
The 10th token has dot product between original and final of 112.06
On the other hand, this is what the dot product of two random vectors looks like
The average absolute value of the dot pro

# Data prep


These are the columns in the data set:

precursor_mz - f64
precursor_charge - f64
mzs - list[f64]
intensities - list[f64]
in_silico - bool
smiles - str
adduct - str
collision_energy - str
instrument_type - str
compound_class - str
entropy - f64
scaffold_smiles - str

In [ ]:
# import the data (with pandas?)
import pandas as pd

## Load the dataset (for some reason this didn't work for me)
#df = pd.read_parquet('enveda_library_subset 2.parquet')

#print(df.head())


# tokenize the SMILES. Do we need to pad? If so, what's the max length
def tokenize_function(examples):
    return tokenizer(examples['smiles'], truncation=True, padding='max_length', max_length=128)

# custom Dataset class for all the types of data.
# I think we might want to make a new 'column' of data that combines mzs and intensities into "label"

from torch.utils.data import Dataset

class SMILESDataset(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.tokenizer = tokenizer
        self.smiles = dataframe['smiles'].tolist()
        self.precursor_mz = dataframe['precursor_mz'].tolist()
        self.precursor_charge = dataframe['precursor_charge'].tolist()
        self.collision_energy = dataframe['collision_energy'].tolist()
        self.instrument_type = dataframe['instrument_type'].tolist()
        self.in_silico_label = dataframe['in_silico_label'].tolist()
        self.adduct = dataframe['adduct'].tolist()
        self.compound_class = dataframe['compound_class'].tolist()
        self.mzs = dataframe['mzs'].tolist()
        self.intensities = dataframe['intensities'].tolist()

        # Create labels as a 2D array of mzs and intensities put together. Or have it flat and just concat both
        #self.labels = #TODO

        # Create supplementary data as a long concatinated list of all the supplementary data
        #self.supplementary_data = #TODO

    def __len__(self):
        return len(self.smiles)

    def __getitem__(self, idx):
        smiles = self.smiles[idx]
        precursor_mz = self.precursor_mz[idx]
        label = self.labels[idx]

        # Tokenize SMILES
        inputs = self.tokenizer(smiles, truncation=True, padding='max_length', max_length=128, return_tensors='pt')

        # Prepare item
        item = {key: val.squeeze(0) for key, val in inputs.items()}  # Remove batch dimension
        item['precursor_mz'] = torch.tensor(precursor_mz, dtype=torch.float)
        item['labels'] = torch.tensor(label, dtype=torch.long)

        return item

# test/train split
# Use Murcko scaffold and spectral entropy splitting for this, rather than random.
# This will ensure that similar molecules don't go into both training and test,
# causing cross contamination and over fitting.

def split_data(df):
    # implement something not random here
    return train_test_split(df, test_size=0.1, random_state=42)

# train_df, eval_df = split_data(df)

# train_dataset = SMILESDataset(train_df, tokenizer)
# eval_dataset = SMILESDataset(eval_df, tokenizer)

## batch?


ArrowInvalid: Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.

# Custom model for our problem
This is probably the most important part in terms of design choices. We are changing the ChemBERTa model by adding on something at the end. This new module will take the hidden SMILES embedding from the last hidden layer as input. It will also take in all the other data about the precusor molecule and experimental conditions (eg, precusor mz, collison energy etc). For now, let's call that supplementary data.

I've written the simplest possible thing here: a single linear layer that takes the embedding of the entire seq, concatinated with all the supplementary data for the example. It outputs "labels", which is mzs and intensities zipped together.

The reason for making a single module output both mzs and intensities is because there needs to be the same number of fragments per example, and the two numbers are very related.

A single linear layer is probably a terrible choice though, since this is the only layer that sees all the supplementary data.

In [ ]:
import einops

max_fragments = 10 #find out what the maximum number of fragments is in the data

class CustomChemBERTaModel(nn.Module):
    def __init__(self, model):
        super(CustomChemBERTaModel, self).__init__()
        self.model = model

        # Get hidden size from the ChemBERTa model configuration
        hidden_size = self.model.config.hidden_size
        seq_length = 128 #Should we set a max sequence length and pad like this?

        # Define a linear layer to output 2 numbers (mz and intensity) per fragment
        self.linear = nn.Linear(hidden_size * seq_length, 2 * max_fragments)

    def forward(self, input_ids, supplementary_data=None, labels=None):
        # Pass inputs through ChemBERTa
        outputs = self.model(input_ids=input_ids, output_hidden_states=True)

        # Extract last hidden state (embeddings)
        last_hidden_state = outputs.hidden_state[-1]
        flatten_hidden_state = einops.rearrange(last_hidden_state, 'b s h -> b (s h)')

        # Pass in supplementary data and then contat it with flatten_hidden_state
        # state = contat(flatten_hidden_state, supplementary_data) #TODO

        # Pass through the linear layer
        predicted_output_flat = self.linear(state)  # Shape: [batch_size, 2 * max_fragments]
        predicted_output = einops.rearrange(predicted_output_flat, 'b (h f) -> b h f', f=max_fragments)

        # calculate the loss by comparing to labels
        loss = 0 #TODO use the loss function they mentioned on discord (I think dot product similarity)

        return predicted_output, loss

#MS_model = CustomChemBERTaModel(model, supplementary_data, labels)

# LoRA config


In [ ]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["key", "query", "value"] # they seem to drop off the "key" often?
    modules_to_save=["classifier"] # change this to the name of the new modules at the end.
    bias="none"
)

peft_model = get_peft_model(model, peft_config)

# I don't think this is stickly necessary?
# In fact, may even be bad since it might freeze params in our last layer:
for param in peft_model.base_model.parameters():
    param.requires_grad = False

model = get_peft_model(model, config)
model.print_trainable_parameters() #check that it's training the right things

# Training the Model

In [ ]:
from torch.optim import AdamW
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=data_collator)
optimizer = AdamW(peft_model.parameters(), lr=5e-5)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
peft_model.to(device)
peft_model.train()

for epoch in range(training_args.num_train_epochs):
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        supplementary_data = batch['supplementary_data'].to(device)
        labels = batch['labels'].to(device)

        outputs = peft_model(
            input_ids=input_ids,
            supplementary_data=supplementary_data,
            labels=labels
        )
        loss = outputs[1]

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


# Inference

In [ ]:
def prepare_inference_input(smiles, precursor_mz):
    inputs = tokenizer(smiles, truncation=True, padding='max_length', max_length=128, return_tensors='pt')
    inputs = {key: val.to(device) for key, val in inputs.items()}
    inputs['supplementary_data'] = torch.tensor([supplementary_data], dtype=torch.float).to(device)
    return inputs

peft_model.eval()

# Example data
smiles_example = "CCO"
supplementary_data_example = 0 #TODO

# Prepare input
inputs = prepare_inference_input(smiles_example, supplementary_data_example)

# Inference
with torch.no_grad():
    outputs = peft_model(**inputs)
    logits = outputs[0]


# Choices that affect the whole architecture

*   Format for the supplementary data
*   Format for the label data
*   The format of the output of the new model



### More modular choices (that are important)


*   Whether we have to predict compound_class at inference
*   Include in_silico data?
*   Architeture of the modified ChemBERTa model
*   LoRA parameters


